In [1]:
# Initialize Otter
import otter
grader = otter.Notebook("lab.ipynb")

# Lab 8 – Feature Engineering ⚙️

## DSC 80, Spring 2025

### Due Date: Wednesday, May 28th at 11:59pm

## Instructions

Welcome to the eight DSC 80 lab this quarter!

Much like in DSC 10, this Jupyter Notebook contains the statements of the problems and provides code and Markdown cells to display your answers to the problems. Unlike DSC 10, the notebook is *only* for displaying a readable version of your final answers. The coding will be done in an accompanying `lab.py` file that is imported into the current notebook, and **you will only submit that `lab.py` file**, not this notebook!

Some additional guidelines:
- **Unlike in DSC 10, labs will have both public tests and hidden tests.** The bulk of your grade will come from your scores on hidden tests, which you will only see on Gradescope after the assignment deadline.
- **Do not change the function names in the `lab.py` file!** The functions in the `lab.py` file are how your assignment is graded, and they are graded by their name. If you changed something you weren't supposed to, you can find the original code in the [course GitHub repository](https://github.com/dsc-courses/dsc80-2025-sp).
- Notebooks are nice for testing and experimenting with different implementations before designing your function in your `lab.py` file. You can write code here, but make sure that all of your real work is in the `lab.py` file, since that's all you're submitting.
- You are encouraged to write your own additional helper functions to solve the lab, as long as they also end up in `lab.py`.

**To ensure that all of the work you want to submit is in `lab.py`, we've included a script named `lab-validation.py` in the lab folder. You shouldn't edit it, but instead, you should call it from the command line (e.g. the Terminal) to test your work.** More details on its usage are given at the bottom of this notebook.

**Importing code from `lab.py`**:

* Below, we import the `.py` file that's contained in the same directory as this notebook.
* We use the `autoreload` notebook extension to make changes to our `lab.py` file immediately available in our notebook. Without this extension, we would need to restart the notebook kernel to see any changes to `lab.py` in the notebook.
    - `autoreload` is necessary because, upon import, `lab.py` is compiled to bytecode (in the directory `__pycache__`). Subsequent imports of `lab` merely import the existing compiled python.

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from lab import *

In [4]:
import pandas as pd
import numpy as np
import plotly.express as px
import statsmodels.api as sm
from pathlib import Path
from sklearn.preprocessing import Binarizer, QuantileTransformer, FunctionTransformer

import warnings
warnings.filterwarnings('ignore')

## Part 1: Scaling Transformations 📐

A scaling transformation transforms the scale of the data of a particular quantitative column. Mathematically, each data point $d_i$ is replaced with a transformed value $t_i = f(d_i)$, where $f$ is a transformation function. We can transform any column in a dataset, whether it corresponds to a feature or a target.

Generally, the goal of a scaling transformation is to change the data from a complicated, non-linear relationship into a **linear** relationship. Linear relationships are very easy to understand and are easily used by models, like linear regression.

However, non-linear growth is a commonly seen relationship in data. Sometimes this growth is by a **fixed power** and sometimes it is **exponential**. The scaling transformations that turn these types of growth linear are **root** and **log** transformations respectively. (Generally, it is more difficult to determine which transformation is appropriate for a given dataset, though the [Tukey-Mosteller bulge diagram](https://freakonometrics.hypotheses.org/files/2014/06/Selection_005.png) is useful.)

Let's start by looking at some examples of transformations.

#### Example 1

Run the cell below to generate a scatter plot.

In [5]:
# By setting a seed, we guarantee that we will see the same results each time we run this cell.
np.random.seed(23)

# Generates a random scatter plot
x = np.arange(1, 101) + np.random.normal(0, 0.5, 100)
y = 2 * ((x + np.random.normal(0, 1, 100)) ** 2) + np.abs(x) * np.random.normal(0, 30, 100)
df_1 = pd.DataFrame().assign(x=x, y=y)

px.scatter(df_1, x='x', y='y', trendline="ols", trendline_color_override="red")

It doesn't appear to be the case that `'x'` and `'y'` are linearly associated here, and they aren't – there is a **quadratic** relationship between them. Note that if we were to create a **residual plot** above, there would be a pattern – the residuals for smaller `'x'` would mostly be positive, and the residuals for larger `'x'` would mostly be negative. From [DSC 10](https://inferentialthinking.com/chapters/15/5/Visual_Diagnostics.html), we know that patterns in a residual plot imply that the relationship between the two variables is non-linear.

To linearize the relationship, we can take the square root of each `'y'` value:

In [6]:
df_1['root y'] = np.sqrt(df_1['y'])

px.scatter(df_1, x='x', y='root y', trendline="ols", trendline_color_override="red")

That looks much better!

#### Example 2

Run the cell below to generate another scatter plot.

In [7]:
# By setting a seed, we guarantee that we will see the same results each time we run this cell
np.random.seed(32)

# Generates a different random scatter plot
x = np.linspace(2, 5, 100)
y = 10 * (np.e ** x) + np.abs(x) * np.random.normal(0, 5, 100) + np.random.normal(0, 30, 100)
df_2 = pd.DataFrame().assign(x=x, y=y)

px.scatter(df_2, x='x', y='y', trendline="ols", trendline_color_override="red")

Again, the relationship between `'x'` and `'y'` is not quite linear. Let's try the square root transformation we tried in Example 1:

In [8]:
df_2['root y'] = np.sqrt(df_2['y'])

px.scatter(df_2, x='x', y='root y', trendline="ols", trendline_color_override="red")

Hmm... the relationship certainly looks _more_ linear than before, but still not quite linear. Let's look at the residual plot:

In [9]:
# Feel free to use this function directly to help you answer Question 1.
def create_residual_plot(df, x, y):
    df = df.copy()
    from sklearn.linear_model import LinearRegression
    model = LinearRegression()
    model.fit(df[[x]], df[y])
    df['pred'] = model.predict(df[[x]])
    df[f'{y} residuals'] = df[y] - model.predict(df[[x]])
    return px.scatter(df, x='pred', y=f'{y} residuals', trendline='ols', trendline_color_override='red')

create_residual_plot(df_2, 'x', 'root y')

There is clearly a pattern in the residual plot. Let's instead try another transformation for the `'y'` values – $\log$.

In [10]:
df_2['log y'] = np.log(df_2['y'])

px.scatter(df_2, x='x', y='log y', trendline="ols", trendline_color_override="red")

That looks much better! We can verify that the residual plot has no "patterns":

In [11]:
create_residual_plot(df_2, 'x', 'log y')

Note – there is still evidence of **heteroscedasticity**, or "uneven spread", in this scatter plot, but the relationship is as close to linear as we'll get.

### Question 1 –  Root vs. Log

Now that we've learned how to perform transformations with example datasets, it's your job to apply these ideas to a real dataset. Below, you are given a dataset that describes the [number of home runs in the MLB per year](https://www.mlb.com/glossary/standard-stats/home-run). The relationship between the two variables, `'Year'` and `'Homeruns'`, is not linear.

**Specifically, your job is to determine what the appropriate transformation to apply to the `'Home runs'` column is, in order to linearize the relationship.**

Complete the implementation of the function `best_transformation`, which returns either 1, 2, 3, or 4, with the value corresponding to one of the following choices:

1. Square root transformation.
2. Log transformation.
3. Both work the same.
4. Neither gives a transformation revealing a linear relationship.

***Hint***: If you find that both residual plots have some sort of pattern, choose the residual plot in which the vertical spread is constant. There is one clearly correct answer.

In [12]:
homeruns_fp = Path('data')/'homeruns.csv'
homeruns = pd.read_csv(homeruns_fp)

In [22]:
homeruns
# px.scatter(homeruns, x='Year', y='Homeruns', trendline="ols", trendline_color_override="red")

new_homeruns = homeruns.copy()
new_homeruns['Sqrt Homeruns'] = np.sqrt(new_homeruns['Homeruns'])
new_homeruns['Log Homeruns'] = np.log(new_homeruns['Homeruns'])
new_homeruns

px.scatter(new_homeruns, x='Year', y='Log Homeruns', trendline="ols", trendline_color_override="red")
create_residual_plot(new_homeruns, 'Year', 'Sqrt Homeruns')

In [24]:
grader.check("q1")

q1 results: All test cases passed!

## Part 2: Diamond Pricing 💎

In this next section, you will pretend you are a jewelry appraiser and predict the prices of diamonds given several standard characteristics of diamonds.

You will use linear regression to predict prices, while improving the quality of your predictions using **feature engineering**. Since this question is supposed to help you understand feature engineering, **you will be building these features from scratch, instead of using the built in `sklearn` or `pandas` methods**.

The `diamonds` dataset is accessible via `seaborn` (with `sns.load_dataset('diamonds')`), but we've skipped that step and loaded it for you below. The DataFrame has 53940 rows and 10 columns:

|column|description|unique values or range|
|---|---|---|
|`'carat'`|weight of the diamond in carats (each carat is 0.2 grams)| 0.2 - 5.01 |
|`'cut'`|quality of the cut | Fair, Good, Very Good, Premium, Ideal |
|`'color'`|diamond colour | J (worst, near colorless), I, H, G, F, E, D (best, absolute colorless) |
|`'clarity'`|a measurement of how clear the diamond is | I1 (worst), SI2, SI1, VS2, VS1, VVS2, VVS1, IF (best) |
|`'depth'`|total depth percentage, computed as z / mean(x, y) = 2 * z / (x + y) | 43 - 79 |
|`'table'`|width of top of diamond relative to widest point | 43 - 95 |
|`'price'`|price in US dollars | \\$326 - \\$18,823 USD |
|`'x'`|length in mm | 0 - 10.74 |
|`'y'`|width in mm | 0 - 58.9 | 
|`'z'`|depth in mm | 0 - 31.8 |

If you want to learn more about how diamonds are measured, refer to [this page by the American Gem Society](https://www.americangemsociety.org/4cs-of-diamonds/).

In [25]:
diamonds = pd.read_csv(Path('data')/'diamonds.csv')
diamonds.head()

,carat,cut,color,clarity,depth,table,price,x,y,z
0,0.23,Ideal,E,SI2,61.5,55.0,326,3.95,3.98,2.43
1,0.21,Premium,E,SI1,59.8,61.0,326,3.89,3.84,2.31
2,0.23,Good,E,VS1,56.9,65.0,327,4.05,4.07,2.31
3,0.29,Premium,I,VS2,62.4,58.0,334,4.20,4.23,2.63
4,0.31,Good,J,SI2,63.3,58.0,335,4.34,4.35,2.75


### Question 2 – Ordinal Encoding 🔢

Every categorical variable in the dataset is an ordinal column, meaning that there is an inherent order that we can use to sort the values in the column. Recall that **ordinal encoding** is a feature transformation that maps the values in an ordinal column to positive integers in a way that preserves the order of the column values. For instance, an ordinal encoding for Freshman, Sophomore, Junior, Senior is 0, 1, 2, 3.

Complete the implementation of the function `create_ordinal`, which takes in the `diamonds` DataFrame and returns a DataFrame of ordinal features only with names of the form `'ordinal_<col>'`, where `'<col>'` is the original categorical column name. For instance, the `'ordinal_color'` column should consist of values from 0 to 6, where 0 refers to `'J'` and 6 refers to `'D'`. (In all cases, start counting from 0.)

***Notes:*** 
- Remember, you are creating this function using basic `pandas`. You might want to create a helper function that takes in a single column and an ordering for that column.
- Don't include non-ordinal features in the returned DataFrame. That is, if there are only three columns in `diamonds` that are ordinal, `create_ordinal` should return a DataFrame with three columns.
- The orderings for each of the ordinal columns are displayed in the data dictionary above (in the `'unique values or range'` column).

In [ ]:
diamonds

diamond_dct = {'Fair': 0, 
        'Good': 1,
        'Very Good': 2,
        'Premium': 3, 
        'Ideal': 4,
        'J': 0,
        'I': 1,
        'H': 2,
        'G': 3,
        'F': 4,
        'E': 5,
        'D': 6,
        'I1': 0,
        'SI2': 1,
        'SI1': 2,
        'VS2': 3,
        'VS1': 4,
        'VVS1': 5,
        'VVS2': 6
        }

def helper(df, col_name):

    df[f"ordinal_{col_name}"] = df[col_name].apply(lambda x: diamond_dct[x])
    df.drop(col_name, axis=1)


new_diamonds = diamonds.copy()

for i in new_diamonds.columns:
    if new_diamonds[i].iloc[0] in diamond_dct:
        helper(new_diamonds, i)
    else:
        new_diamonds.drop(i, axis=1)

new_diamonds


SyntaxError: invalid syntax (4050058526.py, line 27)

In [26]:
# don't change this cell, but do run it -- it is needed for the tests to work
diamonds = pd.read_csv(Path('data')/'diamonds.csv')
out_q2 = create_ordinal(diamonds)

In [ ]:
grader.check("q2")

### Question 3 – Nominal Encoding 📊

Even though the categorical variables in the dataset are ordinal, we can still treat them as nominal by forgetting their order. To treat the categorical variables in our dataset as nominal, we might **one-hot encode** them. 

#### `create_one_hot`

Complete the implementation of the function `create_one_hot`, which takes in the `diamonds` DataFrame and returns a DataFrame of one-hot encoded features with names of the form `'one_hot_<col>_<val>'`, where `'<col>'` is the original categorical column name, and `'<val>'` is the value found in the categorical column `'<col>'`. For instance, one of your column names will be `'one_hot_color_J'`.

***Notes***:
- Only include one-hot-encoded columns in the DataFrame that `create_one_hot` returns.
- Create a helper function that creates the one-hot encoding for a single column. **Do not** use `sklearn` or `pd.get_dummies` for this question!
- As per usual, write an efficient implementation. You may use a `for`-loop over **columns**, but not over rows. And the order of **columns** does not matter.
- In lecture, we will look at cases where we need to drop one one-hot encoded column per categorical variable. **Do not drop** any one-hot encoded columns here!

<br>

#### `create_proportions`

Similar to the one-hot encoding case, you can replace a value in a nominal column with the proportion of times that value appears in the column. For instance, if a column consists of the values `['a', 'b', 'a', 'c']`, then the proportion-encoded column is `[0.5, 0.25, 0.5, 0.25]`.  This might be a reasonable approach to predicting the price of a diamond, as you might expect *rarer attributes to be considered more valuable* than common ones.

Complete the implementation of the function `create_proportions`, which takes in the `diamonds` DataFrame and returns a DataFrame of proportion-encoded features with names of the form `'proportion_<col>'`, where `'<col>'` is the original categorical column name.

In [29]:
# don't change this cell, but do run it -- it is needed for the tests to work
diamonds = pd.read_csv(Path('data')/'diamonds.csv')
out1_q3 = create_one_hot(diamonds)
out2_q3 = create_proportions(diamonds)

In [ ]:
grader.check("q3")

### Question 4 – Quadratic Features 📈

Linear regression doesn't capture non-linear relationships between variables. However, you can create features that encode such dependencies **before** fitting your regression model. Creating polynomial features is one way to do this. For example, the diamonds dataset contains `'x'`, `'y'`, and `'z'` dimensions for each stone. However, different combinations of size may be more valuable than others: a "deep and wide" diamond might be considered more valuable than a shallow, but "long and wide" diamond.

Complete the implementation of the function `create_quadratics`, which takes in the `diamonds` DataFrame and returns a DataFrame of quadratic features of the form `'<col1> * <col2>'`, where `'<col1>'` and `'<col2>'` are the original quantitative columns. The output DataFrame should contain a column for every distinct pair of quantitative columns in `diamonds` (aside from `price`, which should be left out as it is what we are predicting). For instance, one of the columns in the returned DataFrame should named either `'carat * x'` or `'x * carat'`; the order of column names is not important.

***Notes***:
- Again, **do not** use `sklearn` for this question! 
- Try finding all pairs of quantitative columns efficiently; don't use a nested loop (hint: you may `import itertools`). Our solution contains just a single `for`-loop (over pairs of columns).
- If you import itertools, you must include the itertools import in both this notebook and in `lab.py`.
- The columns of the resulting DataFrame may be in any order.

In [42]:
# don't change this cell, but do run it -- it is needed for the tests to work
diamonds = pd.read_csv(Path('data')/'diamonds.csv')
out_q4 = create_quadratics(diamonds)

In [ ]:
grader.check("q4")

### Question 5 – Comparing Performance 🏆

We've now created several sets of features. **Which features are best able to predict the price of a diamond in a linear regression model?** We'll look at both single-feature linear regression models and multiple regression models. In all cases, use the default arguments to `sklearn`'s `LinearRegression` object (i.e. assume there is an intercept term).

<br>

Two of the evaluation metrics we can use to compare regression models are RMSE and $R^2$.

RMSE, or Root Mean Squared Error, roughly measures the average distance between a model's predicted values and the actual values. The RMSE indicates how concentrated predictions are around the line of best fit, with lower values typically indicating smaller residuals and better model performance. RMSE can be calculated using the formula below, where $\text{n}$ is the number of data points, $\hat{y_i}$ is the $i$th prediction, and $y_i$ is the corresponding actual value.

$$\text{RMSE} = \sqrt{\frac{\sum_{i=1}^n (y_i - \hat{y_i})^2}{n}}$$

Remember that in least squares regression, we choose the model parameters that minimize RMSE.

<br>

$R^2$, also known as the coefficient of determination, denotes the proportion of variance in the dependent variable that the independent variable explains. It is used to assess the quality of a linear fit, and in the simple linear regression case, it is the square of the correlation coefficient $r$. $R^2$ values range between 0 and 1, with lower values indicating a worse linear fit and with higher values indicating a better linear fit. If `model` is a fitted `sklearn` regression object, $R^2$ can be computed by calling `model.score`. 

#### Single-Feature Models

- (1) Fit a single-feature linear regression model on `'carat'`. What is the $R^2$ of the model? (Note that `'carat'` turns out to be the best single feature to use in a linear model that predicts price.)
- (2) What is the RMSE of the model you created in (1)?
- (3) Amongst the other **quantitative** features present in the original `diamonds`, which produces the single-feature linear regression model with the highest $R^2$?
- (4) Amongst all the new features you created in Questions 2-4, which produces the single-feature linear regression model with the highest $R^2$?
- (5) Amongst the new categorical features you created in Question 2 and 3, which produces the single-feature linear regression model with the highest $R^2$? 

#### Multiple Regression

Now, fit a multiple regression model using:
- the quantitative columns that were present in the original `diamonds` dataset, and
- the quantitative features engineered in Question 4

as features. (Don't use any of the encodings of categorical columns from Questions 2 and 3.)

- (6) What is the RMSE of this new model?


<br>

Complete the implementation of the function `comparing_performance`, which returns a list containing the answers to the 6 questions numbered (1), (2), ..., (6) above. You don't need to round any of your answers.

***Hint:*** Repeatedly use the `sklearn` pattern included below. It's a good idea to make a helper function that takes in a column, performs single-feature regression using the input column as the feature, and returns the $R^2$ and RMSE of the model.

***Note:*** The code you submit should be a hard-coded list.

In [53]:
from sklearn.linear_model import LinearRegression

# X = ...
# y = ...

# lr = LinearRegression()
# lr.fit(X, y)  # X is a DataFrame of training data; y is a Series of prices
# lr.score(X, y)  # R-squared
# lr.predict(X) # predicted prices

In [54]:
# don't change this cell, but do run it -- it is needed for the tests to work
import numbers
out_q5 = comparing_performance()

In [ ]:
grader.check("q5")

## Part 3 – Feature Engineering with `sklearn` 🧠

In this final question, you will use `sklearn`'s transformers and estimators for feature engineering. While everything you do with `sklearn` is possible to do with `pandas`, `sklearn` transformers enable you to couple your feature engineering with your modeling. This will allow you to more quickly build and assess your models in `sklearn`.

Specifically, you will create a `TransformDiamonds` class that contains the three methods specified below – `transform_carat` (6.1), `transform_to_quantile` (6.2), and `transform_to_depth_pct` (6.3). In the starter code, there is a skeleton for `TransformDiamonds` that is initialized with a DataFrame `diamonds`.

Each of the methods you implement in the `TransformDiamonds` class should take in a DataFrame, initialize a specific `sklearn.Transformer` object (like `Binarizer` or `FunctionTransformer`), and use the transformer to transform columns from the input DataFrame. You should **not** use DataFrame methods like `apply` in this problem.

In [67]:
from sklearn.preprocessing import Binarizer, QuantileTransformer, FunctionTransformer

Question 6 is made up of the three subparts below.

### Question 6.1 – Transforming a Quantitative Column into a Binary Column (`transform_carat`)

We call a diamond **large** if its weight is strictly greater than 1 carat. We want to **binarize** weights, so that they are 1 for large diamonds and 0 for small diamonds. Complete the implementation fo the method `transform_carat`, which takes in a DataFrame like `diamonds` and returns a binarized **array** of weights. Use a `Binarizer` object as your transformer.

***Notes***:
- You will return an array, not a Series, because `sklearn` thinks in terms of `np.ndarray`s, not DataFrames.
- The implementation of this function should only take two lines.

In [69]:
# don't change this cell, but do run it -- it is needed for the tests to work
diamonds = pd.read_csv(Path('data')/'diamonds.csv')
q6a_trans = TransformDiamonds(diamonds)
q6a_out = q6a_trans.transform_carat(diamonds)

In [ ]:
grader.check("q6.1")

### Question 6.2 – Transforming a Quantitative Column into Quantiles (`transform_to_quantile`)

You will now transform the `'carat'` column so that each diamond's weight in carats is replaced with the **percentile** amongst all diamonds in which its weight lies.

Complete the implementation of the method `transform_to_quantiles`, which takes in a DataFrame like `diamonds` and returns an array containing the percentiles of the weight of each diamond, amongst all diamonds in `self.data`. This array should consist of proportions between 0 and 1; for instance, 0.65 will refer to the 65th percentile. The relevant transformer is `QuantileTransformer`. 

Some guidance:

- Unlike with `Binarizer`, you need to `fit` your `QuantileTransformer` before calling `transform` on the input DataFrame `data`. 
    - You should `fit` your transformer on the DataFrame `self.data`, but you should only `transform` the `data` that is passed to `transform_to_quantiles`. 
    - Note that these two DataFrames, `self.data` and `data`, don't have to be the same! For instance, in the last two lines of the testing setup cell below, we fit a `QuantileTransformer` using just the first 1000 rows of `diamonds`, and then `transform` the entire `diamonds` DataFrame. Make sure your `transform_to_quantiles` method works in such a case.
- When initializing your `QuantileTransformer`, use `n_quantiles=100`.

In [78]:
# don't change this cell, but do run it -- it is needed for the tests to work
q6b_trans = TransformDiamonds(diamonds)
q6b_out = q6b_trans.transform_to_quantile(diamonds)
q6b_trans_top_1000 = TransformDiamonds(diamonds[:1000])
q6b_out_top_1000 = q6b_trans_top_1000.transform_to_quantile(diamonds)

In [ ]:
grader.check("q6.2")

### Question 6.3 – Transforming a Quantitative Column Using a Formula

Recall from the introduction to Part 2 that the "depth percentage" of a diamond is defined as

$$\text{Depth Pct.} = 100\% \cdot \frac{2z}{x + y}$$

where $x$, $y$, and $z$ come from the `'x'`, `'y'`, and `'z'` columns in `diamonds`.

Let's suppose that for some reason we don't have access to the `'depth'` column in `diamonds`, and instead need to recreate it just by looking at the `'x'`, `'y'`, and `'z'` columns. 

Complete the implementation of the method `transform_to_depth_pct`, which takes in a DataFrame like `diamonds` and returns an array consisting of the depth percentages of each diamond. Percentages should be between 0 and 100. The relevant transformer is `FunctionTransformer`.

***Notes***:
- To use `FunctionTransformer`, you will need to define your own function that takes in a 2D array and returns a single array.
- Ignore `ZeroDivisionError` errors, and leave `np.NaN`s as is.
- To verify your work, compare your outputted array to the actual `'depth'` column in `diamonds`.
- It may seem like `FunctionTransformer` is totally unnecessary, since we can compute depth percentages using broadcasting directly. However, as we will see in lecture, transformers can be **pipelined** with other processing steps which greatly simplifies our code.

In [88]:
# don't change this cell, but do run it -- it is needed for the tests to work
diamonds = pd.read_csv(Path('data')/'diamonds.csv')
q6c_trans = TransformDiamonds(diamonds)
q6c_out = q6c_trans.transform_to_depth_pct(diamonds)

In [ ]:
grader.check("q6.3")

## Congratulations! You're done Lab 8! 🏁

As a reminder, all of the work you want to submit needs to be in `lab.py`.

To ensure that all of the work you want to submit is in `lab.py`, we've included a script named `lab-validation.py` in the lab folder. You shouldn't edit it, but instead, you should call it from the command line (e.g. the Terminal) to test your work.

Once you've finished the lab, you should open the command line and run, in the directory for this lab:

```
python lab-validation.py
```

**This will run all of the `grader.check` cells that you see in this notebook, but only using the code in `lab.py` – that is, it doesn't look at any of the code in this notebook. If all of your `grader.check` cells pass in this notebook but not all of them pass in your command line with the above command, then you likely have code in your notebook that isn't in your `lab.py`!**

You can also use `lab-validation.py` to test individual questions. For instance,

```
python lab-validation.py q1 q2 q4
```

will run the `grader.check` cells for Questions 1, 2, and 4 – again, only using the code in `lab.py`. [This video](https://www.loom.com/share/0ea254b85b2745e59322b5e5a8692e91?sid=5acc92e6-0dfe-4555-9b6a-8115b6a52f99) how to use the script as well.

Once `python lab-validation.py` shows that you're passing all test cases, you're ready to submit your `lab.py` (and only your `lab.py`) to Gradescope. Once submitting to Gradescope, make sure to stick around until all test cases pass.

There is also a call to `grader.check_all()` below in _this_ notebook, but make sure to also follow the steps above.

---

To double-check your work, the cell below will rerun all of the autograder tests.

In [ ]:
grader.check_all()